In [100]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.linear_model import Lasso

## Preparando os dados

In [135]:
df = pd.read_csv('quinto_andar.csv')

In [136]:
df.head()

,url,bairro,aluguel,condominio,iptu,seguro_incendio,taxa_serviço,total,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox
0,https://www.quintoandar.com.br/imovel/89332996...,Liberdade,R$ 1.900,R$ 424,R$ 5,R$ 25,R$ 38,R$ 2.392,57 m²,1 quarto,1 banheiro,1 vaga,18º andar,Aceita pet,Sem mobília,Metrô próx.
1,https://www.quintoandar.com.br/imovel/89334069...,Santa Cecília,R$ 1.190,R$ 460,R$ 0,R$ 16,R$ 23,R$ 1.689,56 m²,1 quarto,1 banheiro,Sem vaga,11º andar,Aceita pet,Mobiliado,Metrô próx.
2,https://www.quintoandar.com.br/imovel/89333859...,Aclimação,R$ 2.330,R$ 450,R$ 5,R$ 30,R$ 46,R$ 2.861,48 m²,1 quarto,1 banheiro,1 vaga,10º andar,Aceita pet,Sem mobília,Metrô próx.
3,https://www.quintoandar.com.br/imovel/89337065...,Liberdade,R$ 1.850,R$ 418,R$ 6,R$ 24,R$ 37,R$ 2.335,44 m²,1 quarto,1 banheiro,1 vaga,3º andar,Aceita pet,Mobiliado,Metrô próx.
4,https://www.quintoandar.com.br/imovel/89337700...,Vila Guilherme,R$ 2.200,R$ 392,R$ 10,R$ 28,R$ 44,R$ 2.674,62 m²,2 quartos,2 banheiros,1 vaga,8º andar,Aceita pet,Sem mobília,Não próx.


In [137]:
df['quarto'].unique()

array(['1 quarto', '2 quartos', '3 quartos', '4 quartos', '5 quartos',
       'Vila Olímpia, São Paulo', '0 quarto'], dtype=object)

In [138]:
df.loc[df['quarto'] == 'Vila Olímpia, São Paulo']

,url,bairro,aluguel,condominio,iptu,seguro_incendio,taxa_serviço,total,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox
1562,https://www.quintoandar.com.br/imovel/89335300...,Vila Olímpia,R$ 2.750,R$ 700,R$ 303,R$ 35,R$ 55,R$ 3.843,"Vila Olímpia, São Paulo","Vila Olímpia, São Paulo","Itaim Bibi, São Paulo","Jardim Paulista, São Paulo","Vila Nova Conceição, São Paulo",90 m²,2 quartos,1 banheiro


In [139]:
df = df.drop(df[df['vaga_carro']=='Jardim Paulista, São Paulo'].index)

In [140]:
df['quarto'].unique()

array(['1 quarto', '2 quartos', '3 quartos', '4 quartos', '5 quartos',
       '0 quarto'], dtype=object)

In [141]:
df.drop(['url','total','seguro_incendio','taxa_serviço'], axis=1,inplace=True)

In [142]:
df['aluguel'] = df['aluguel'].str.slice(2)
df['condominio'] = df['condominio'].str.slice(2)
df['iptu'] = df['iptu'].str.slice(2)
df['quarto'] = df['quarto'].str.slice(0,1)
df['metragem'] = df['metragem'].str.slice(0,-2)
df['banheiro'] = df['banheiro'].str.slice(0,1)
df['andar'] = df['andar'].str.slice(0,-7)
df['vaga_carro'] = df['vaga_carro'].str.slice(0,1)
df['aluguel'] = df['aluguel'].str.replace('.','')
df['vaga_carro'] = df['vaga_carro'].str.replace('S','0')
df['condominio'] = df['condominio'].str.replace('cluso','0')
df['condominio'] = df['condominio'].str.replace('m info','0')
df['iptu'] = df['iptu'].str.replace('cluso','0')

C:\Users\Douglas\AppData\Local\Temp/ipykernel_11780/2968912937.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['aluguel'] = df['aluguel'].str.replace('.','')


In [143]:
df['aceita_pet'] = df['aceita_pet'].str.replace('Aceita pet','1')
df['aceita_pet'] = df['aceita_pet'].str.replace('Não aceita','0')
df['mobilia'] = df['mobilia'].str.replace('Sem mobília','0')
df['mobilia'] = df['mobilia'].str.replace('Mobiliado','1')
df['metro_prox'] = df['metro_prox'].str.replace('Metrô próx.','1')
df['metro_prox'] = df['metro_prox'].str.replace('Não próx.','0')

C:\Users\Douglas\AppData\Local\Temp/ipykernel_11780/968649670.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['metro_prox'] = df['metro_prox'].str.replace('Metrô próx.','1')
C:\Users\Douglas\AppData\Local\Temp/ipykernel_11780/968649670.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['metro_prox'] = df['metro_prox'].str.replace('Não próx.','0')


In [144]:
df.head()

,bairro,aluguel,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox
0,Liberdade,1900,424,5,57,1,1,1,18,1,0,1
1,Santa Cecília,1190,460,0,56,1,1,0,11,1,1,1
2,Aclimação,2330,450,5,48,1,1,1,10,1,0,1
3,Liberdade,1850,418,6,44,1,1,1,3,1,1,1
4,Vila Guilherme,2200,392,10,62,2,2,1,8,1,0,0


In [145]:
df['andar'].unique()

array(['18', '11', '10', '3', '8', '2', '9', '12', '', '1', '16', '5',
       '13', '6', '4', '24', '7', '15', '14', '17', '20', '21', '19',
       '23', '26', '72', '22', '43', '51', '25', '27'], dtype=object)

In [146]:
df['andar'][df['andar'] == ''] = '1'

In [147]:
df['andar'].unique()

array(['18', '11', '10', '3', '8', '2', '9', '12', '1', '16', '5', '13',
       '6', '4', '24', '7', '15', '14', '17', '20', '21', '19', '23',
       '26', '72', '22', '43', '51', '25', '27'], dtype=object)

In [148]:
df['condominio'] = df['condominio'].astype(float)
df['iptu'] = df['iptu'].astype(float)
df['metragem'] = df['metragem'].astype(float)
df['quarto'] = df['quarto'].astype(int)
df['banheiro'] = df['banheiro'].astype(int)
df['vaga_carro'] = df['vaga_carro'].astype(int)
df['aceita_pet'] = df['aceita_pet'].astype(int)
df['mobilia'] = df['mobilia'].astype(int)
df['metro_prox'] = df['metro_prox'].astype(int)
df['aluguel'] = df['aluguel'].astype(int)
df['andar'] = df['andar'].astype(int)

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2776 entries, 0 to 2776
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bairro      2776 non-null   object 
 1   aluguel     2776 non-null   int32  
 2   condominio  2776 non-null   float64
 3   iptu        2776 non-null   float64
 4   metragem    2776 non-null   float64
 5   quarto      2776 non-null   int32  
 6   banheiro    2776 non-null   int32  
 7   vaga_carro  2776 non-null   int32  
 8   andar       2776 non-null   int32  
 9   aceita_pet  2776 non-null   int32  
 10  mobilia     2776 non-null   int32  
 11  metro_prox  2776 non-null   int32  
dtypes: float64(3), int32(8), object(1)
memory usage: 195.2+ KB


In [150]:
label_bairro = LabelEncoder()

In [151]:
df['bairro'] = label_bairro.fit_transform(df['bairro'])

In [152]:
df['bairro'].unique()

array([16, 24,  0, 28, 21, 22,  1, 12, 18,  8, 15, 20, 10, 26,  9,  3,  6,
        5,  2, 30, 29, 23,  4, 33, 11,  7, 32, 31, 13, 25, 34, 35, 27, 14,
       17, 19])

In [153]:
df.corr()

,bairro,aluguel,condominio,iptu,metragem,quarto,banheiro,vaga_carro,andar,aceita_pet,mobilia,metro_prox
bairro,1.000000,0.223106,-0.028462,0.172343,0.187869,0.172082,0.200855,0.172726,-0.014732,0.007338,0.032572,-0.072717
aluguel,0.223106,1.000000,-0.070732,0.586443,0.638454,0.533279,0.655591,0.575838,0.122908,0.094687,0.203084,-0.109839
condominio,-0.028462,-0.070732,1.000000,-0.217170,-0.175750,-0.100031,-0.146782,0.052334,0.226776,0.053388,0.049727,0.130662
iptu,0.172343,0.586443,-0.217170,1.000000,0.563984,0.469924,0.576221,0.563730,0.029267,0.050353,0.085564,-0.077573
metragem,0.187869,0.638454,-0.175750,0.563984,1.000000,0.727096,0.676115,0.462426,-0.045436,0.142075,-0.042694,-0.143934
quarto,0.172082,0.533279,-0.100031,0.469924,0.727096,1.000000,0.666243,0.456166,-0.003798,0.120401,-0.079092,-0.196894
banheiro,0.200855,0.655591,-0.146782,0.576221,0.676115,0.666243,1.000000,0.525032,0.043195,0.079920,-0.006160,-0.137467
vaga_carro,0.172726,0.575838,0.052334,0.563730,0.462426,0.456166,0.525032,1.000000,0.183891,0.075774,0.125789,-0.143115
andar,-0.014732,0.122908,0.226776,0.029267,-0.045436,-0.003798,0.043195,0.183891,1.000000,0.032404,0.118214,0.126756
aceita_pet,0.007338,0.094687,0.053388,0.050353,0.142075,0.120401,0.079920,0.075774,0.032404,1.000000,-0.134144,0.022590


In [154]:
ss = StandardScaler()

In [155]:
x = df

In [156]:
y = df['aluguel']

In [157]:
x = ss.fit_transform(x)

In [158]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

## Regressão Linear Simples

In [159]:
reg = LinearRegression()

In [160]:
reg.fit(X_train, y_train)

LinearRegression()

In [161]:
p = reg.predict(X_test)

In [162]:
[mae(y_test,p),
mse(y_test,p)]

[5.7048137055985e-13, 6.289502046527898e-25]

## Lasso


In [163]:
lasso.fit(X_train, y_train)

Lasso()

In [164]:
l = lasso.predict(X_test)

In [165]:
[mae(y_test,l),
mse(y_test,l)]

[0.7067563968652667, 0.9290530569050222]